In [ ]:
#redundant codes

In [ ]:
def midpoint(p1,p2):
    print(p1.x)
    print(p1.y)
    return (p1.x+p2.x)/2, (p1.y+p2.y)/2

def check_iris(p_mesh, p_frame):
    left_eye_landmarks =  [33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7]
    right_eye_landmarks = [362, 398, 384, 385, 386, 387, 388, 466, 263, 249, 390, 373, 374, 380, 381, 382]
    
    iris_left_eye = 468
    iris_right_eye = 472
    ph, pw, pc = p_frame.shape 
    gray = cv2.cvtColor(p_frame, cv2.COLOR_BGR2GRAY)
    hleft_pupil = 0
    hright_pupil = 0
    vleft_pupil = 0
    vright_pupil = 0
    for face_landmarks in p_mesh.multi_face_landmarks:
        print(face_landmarks.landmark[33])
        print(face_landmarks.landmark[133])
        left_x,  left_y  =  midpoint(face_landmarks.landmark[33],  face_landmarks.landmark[133])
        
        right_x, right_y =  midpoint(face_landmarks.landmark[362], face_landmarks.landmark[263])
        for idx, lm in enumerate(face_landmarks.landmark):           
            if idx == iris_left_eye:
                print("yes")
                hleft_pupil = lm.x/ (left_x*2-10)
                vleft_pupil = lm.y / (left_y*2-10)
            elif idx == iris_right_eye:
                print("yes")
                hright_pupil = lm.x/ (right_x*2-10)
                vright_pupil = lm.y / (right_y*2-10)
        horizontal_ratio = (hleft_pupil + hright_pupil)/2
        vertical_ratio   = (vleft_pupil + vright_pupil)/2
    print(f"left_x {left_x} left_y {left_y} \n right_x {right_x} right_y {right_y} \n hleft_pupil {hleft_pupil} hright_pupil {hright_pupil} horizontal_ratio {horizontal_ratio}")
    if horizontal_ratio >= 0.65 or horizontal_ratio < 0.4:
        return True
    else:
        return False

In [ ]:
def draw_mask(ph, pw, eye_region, gray):
    mask = np.zeros((ph, pw), np.uint8)
    cv2.polylines(mask, [eye_region], True, 255,2)
    cv2.fillPoly(mask,[eye_region],255)
    eye = cv2.bitwise_and(gray, gray, mask=mask)
    return eye

def draw_max_min(eye_region):
    min_x = np.min(eye_region[:,0])
    max_x = np.max(eye_region[:,0])
    min_y = np.min(eye_region[:,1])
    max_y = np.max(eye_region[:,1])
    
    return min_x, max_x, min_y, max_y

In [1]:
def check_gaze(p_mesh,p_frame, p_check=False, p_center_value=None): 
    
    check_ratio = False
    eps = 1e-10
    
    left_eye_landmarks =  [33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163,7]
    right_eye_landmarks = [362, 398, 384, 385, 386, 387, 388, 466, 263, 249, 390, 373, 374, 380, 381, 382]
    
    ph, pw, pc = p_frame.shape 
    gray = cv2.cvtColor(p_frame, cv2.COLOR_BGR2GRAY)
    
    left_eye_temp = []
    right_eye_temp = []
    
    for face_landmarks in p_mesh.multi_face_landmarks:
        for idx, lm in enumerate(face_landmarks.landmark):
            if idx in left_eye_landmarks:
                left_eye_temp.append((int(lm.x*pw),int(lm.y*ph)))
            elif idx in right_eye_landmarks:
                right_eye_temp.append((int(lm.x*pw),int(lm.y*ph)))
    
    right_eye_region = np.array(right_eye_temp, np.int32)   
    left_eye_region  = np.array(left_eye_temp, np.int32) 
    
    eye_left = draw_mask(ph, pw, left_eye_region,  gray)
    eye_right = draw_mask(ph,pw, right_eye_region, gray)
    
    min_xl, max_xl, min_yl, max_yl =  draw_max_min(left_eye_region)
    min_xr, max_xr, min_yr, max_yr =  draw_max_min(right_eye_region)
    
    gray_eye_left  = eye_left[min_yl: max_yl, min_xl: max_xl]
    gray_eye_right = eye_right[min_yr: max_yr, min_xr: max_xr]
    
    eye_left = cv2.resize(gray_eye_left, None,fx=5,fy=5)
    eye_right = cv2.resize(gray_eye_right, None,fx=5,fy=5)
    
    _, threshold_eye_left = cv2.threshold(gray_eye_left, 70, 255, cv2.THRESH_BINARY)
    _, threshold_eye_right = cv2.threshold(gray_eye_right, 70, 255, cv2.THRESH_BINARY)
    
    height_l, width_l = threshold_eye_left.shape
    height_r, width_r = threshold_eye_right.shape
    
    left_left_side_threshold = threshold_eye_left[0:height_l, 0:int(width_l/2)]
    left_left_side_white = cv2.countNonZero(left_left_side_threshold)
    
    left_right_side_threshold = threshold_eye_left[0:height_l, int(width_l/2):width_l]
    left_right_side_white = cv2.countNonZero(left_right_side_threshold)
    
    
    right_left_side_threshold = threshold_eye_right[0:height_r, 0:int(width_r/2)]
    right_left_side_white = cv2.countNonZero(right_left_side_threshold)
    right_right_side_threshold = threshold_eye_right[0:height_r, int(width_r/2):width_r]
    right_right_side_white = cv2.countNonZero(right_right_side_threshold)

    if left_left_side_white == 0:
        gaze_ratio_left = 0
    elif left_right_side_white ==0:
        gaze_ratio_left = 1
    else:
        gaze_ratio_left = left_left_side_white/(left_right_side_white+left_left_side_white)
    
    if right_left_side_white == 0:
        gaze_ratio_right = 0
    elif right_right_side_white ==0:
        gaze_ratio_right = 1
    else:
        gaze_ratio_right = right_left_side_white/(right_right_side_white+right_left_side_white)
        
    gaze_ratio = (gaze_ratio_left+gaze_ratio_right)/2
    
    if p_check:
        return gaze_ratio
    else:
        if p_center_value is not None:
            if gaze_ratio < p_center_value * 0.9 or gaze_ratio > p_center_value*1.1: 
                check_ratio = True
        else:
            if gaze_ratio < 0.4 or gaze_ratio > 0.58: # <0.45 right >0.58 left
                check_ratio =  True
                
    return check_ratio

In [2]:
def draw_lines(p_frame, point1, point2, point3): #frame,nose, reye, leye, mbrow
    
    frames = cv2.line(p_frame, (int(point1[0]), int(point1[1]) ), (int(point2[0]), int(point2[1]) ), (255,0,0), 1)
    frames = cv2.line(frames,  (int(point1[0]), int(point1[1]) ), (int(point3[0]), int(point3[1]) ), (255,0,0), 1)
    frames = cv2.line(frames,  (int(point2[0]), int(point2[1]) ), (int(point3[0]), int(point3[1]) ), (255,0,0), 1)
    
    return frames
      
def find_euc_distance(point1,point2):
    distance = math.sqrt(((point1[0] - point2[0]) ** 2) + ((point1[1] - point2[1]) ** 2)) 
    return distance

def area_heron(s1, s2, s3):
    s = (s1+ s2 + s3) / 2
    area = math.sqrt(s * (s - s3) * (s - s2) * (s - s1))
    
    return area

def find_area_tri(point1,point2,point3):
    dist_p12p2 =  find_euc_distance(point1, point2) #nose 2 mbrow
    
    dist_p12p3 =  find_euc_distance(point1, point3) #nose 2 eye
    
    dist_p22p3 =  find_euc_distance(point2, point3) #mbrow 2 eye
    
    area = area_heron(dist_p12p2, dist_p12p3,dist_p22p3)
    
    return area
    

In [3]:
def check_face(p_mesh): #returns face count
    face_count = 0
    
    if p_mesh.multi_face_landmarks: # and not p_det.detections:
        face_count = len(p_mesh.multi_face_landmarks)
        
    return face_count

def check_head_pose(p_mesh, p_pose, p_frame, p_check=False, p_vgaze=None, p_hgaze=None):
    
    max_thres = 0.55
    min_thres = 0.45
    head_pose = False
    ph, pw, pc = p_frame.shape
    
    rmost_eye,lmost_eye, nose_tip = 263, 33, 1
    lside, rside, mchin = 93, 323, 200
    mbrow, forehead, lnose_side = 9,10, 48
    rnose_side,chin  = 278, 152

    for face_landmarks in p_mesh.multi_face_landmarks:
        
        for idx, lm in enumerate(face_landmarks.landmark):
            
            if idx == nose_tip:              
                nose_2d = (lm.x * pw, lm.y * ph)  
                nose_3d = (lm.x * pw, lm.y * ph, lm.z * 3000)
                
            elif idx == rmost_eye: #rightmost eye point 
                reye_2d = (lm.x * pw, lm.y * ph)  
                reye_3d = (lm.x * pw, lm.y * ph, lm.z * 3000)
                
            elif idx == lmost_eye: #leftmost eye point
                leye_2d = (lm.x * pw, lm.y * ph)  
                leye_3d = (lm.x * pw, lm.y * ph, lm.z * 3000)
                
            elif idx == mbrow: #middle of brows
                mbrow_2d = (lm.x * pw, lm.y * ph)  
                mbrow_3d = (lm.x * pw, lm.y * ph, lm.z * 3000)
                
            elif idx == lnose_side: #leftmost point of the nose
                lnose_2d = (lm.x * pw, lm.y * ph)  
                lnose_3d = (lm.x * pw, lm.y * ph, lm.z * 3000)
                
            elif idx == rnose_side: #rightmost point of the nose
                rnose_2d = (lm.x * pw, lm.y * ph)  
                rnose_3d = (lm.x * pw, lm.y * ph, lm.z * 3000)
                
            elif idx == mchin: #middle of the chin point
                mchin_2d = (lm.x * pw, lm.y * ph)  
                mchin_3d = (lm.x * pw, lm.y * ph, lm.z * 3000)          
                    
    area_left  = find_area_tri(nose_2d, mbrow_2d, leye_2d) #left eye nose brow
    area_right = find_area_tri(nose_2d, mbrow_2d, reye_2d) #right ey nose brow
    
    area_bottom = find_area_tri(mchin_2d, lnose_2d, rnose_2d) #chin to 
    area_top    = find_area_tri(mbrow_2d, lnose_2d, rnose_2d) # forehead to 
    
    
    horizontal_ratio = area_left / (area_left+area_right)
    #print(ratio)
    
    vertical_ratio = area_bottom / (area_bottom+area_top)
    #print(f"bottom to top: {vertical_ratio}")
    if horizontal_ratio >= vertical_ratio:
        frame =  draw_lines(p_frame, nose_2d, leye_2d, mbrow_2d)
        frame =  draw_lines(frame, nose_2d, reye_2d, mbrow_2d)
    else:
        frame = draw_lines(p_frame, mchin_2d, lnose_2d, rnose_2d)
        frame = draw_lines(frame, mbrow_2d, lnose_2d, rnose_2d)
    label = "center"
    if  horizontal_ratio > max_thres or horizontal_ratio < min_thres:
        head_pose = True 
        label = "left or right"
    if not head_pose:
        if vertical_ratio > 0.5 or vertical_ratio < 0.4:
            head_pose =  True
            label = "down or up"
            
    #test        
    if p_check:
        return horizontal_ratio, vertical_ratio
    else:
        if p_vgaze is not None or p_hgaze is not None:
            if vertical_ratio < p_vgaze * 0.9 or vertical_ratio > p_vgaze * 1.1: 
                head_pose = True
            elif horizontal_ratio < p_hgaze * 0.9 or horizontal_ratio > p_hgaze * 1.1: 
                head_pose = True
        else:
            if horizontal_ratio > max_thres or horizontal_ratio < min_thres:
                head_pose = True
            if not head_pose:
                if vertical_ratio > 0.5 or vertical_ratio < 0.4:
                    head_pose = False
                

    return head_pose #, horizontal_ratio, vertical_ratio

In [ ]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False,max_num_faces=10,min_detection_confidence=0.5, min_tracking_confidence=0.5)

mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

mp_pose = mp.solutions.pose
face_pose = mp_pose.Pose(static_image_mode=True,model_complexity = 1,min_detection_confidence=0.5, min_tracking_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
drawing_spec = mp_drawing.DrawingSpec(thickness=1,circle_radius=1)

path = "C:\\Users\\99455\\Desktop\\DataScience\\Proctoring Task\\Frames\\*.*"
#video_path =  'C:/Users/99455/Desktop/DataScience/OEP database/subject1/Yousef1.avi'
#video_path = "C:\\Users\\99455\\Desktop\\DataScience\\Proctoring Task\\testphoto.jpg"
video_path = "C:\\Users\\99455\\Desktop\\DataScience\\Proctoring Task\\testvideo6.mp4"

#video_path = 0

In [ ]:
3dist_re2nose  =   find_euc_distance(reye_2d, nose_2d)
    dist_le2nose  =   find_euc_distance(leye_2d, nose_2d)
    
    dist_re2mbrow =   find_euc_distance(reye_2d, mbrow_2d)
    dist_le2mbrow =   find_euc_distance(leye_2d, mbrow_2d)
    
    dist_nose2mbrow =   find_euc_distance(nose_2d, mbrow_2d)
    
    n = p_pose.pose_landmarks.landmark[0].presence
    le = p_pose.pose_landmarks.landmark[3].visibility
    re = p_pose.pose_landmarks.landmark[6].visibility

In [ ]:
while cap.isOpened():
    success, frame =  cap.read()
    start =time.time()
    
    
    if not success:
        break #video ends
    frame_no += 1
    #print(frame_no)
    
    if (frame_no % skip_rate == 0):
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) #cv2.flip(frame,1)
        frame.flags.writeable = False
        
        res_mesh = face_mesh.process(frame)
        res_det = face_detection.process(frame)
        res_pose = face_pose.process(frame)
    
        frame.flags.writeable = True
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    
        fw, fh, fc = frame.shape
        face_3d = []
        face_2d = []
        num_face =check_face(res_mesh)
        w,h,c = frame.shape
        
        if num_face == 0:
            #store in No Face Detected -> suspicious
        
            cv2.imwrite("FaceNotDetected/frame_%d.jpg" % frame_no, frame) 
        elif num_face > 1:
            #store in More Than One Face Detected -> suspicious
            cv2.imwrite("MoreThanOneFace/frame_%d.jpg" % frame_no, frame)
        elif num_face == 1:
            #crop frame
            #process further
            #check Extreme Head Pose Detection

            #if bx:
                
            #    cv2.imshow("Head pose", frames)
                #cv2.imwrite("ExtremeHeadPose/frame_%d.jpg" % frame_no, frame)     
            #    cv2.waitKey(0)
                #store in Extreme Head Pose Detected
            #else:
            #    cv2.imshow("Head pose", frames)
            #    cv2.waitKey(0)
                #cv2.imwrite("Center/frame_%d.jpg" % frame_no, frame)
                #Probably only centered images left
                #check Gaze Estimation
                #if check_gaze():
                    #store in Gaze Detected
                #else:
                    #Normal Pictures
            hp = check_head_pose(res_mesh, res_pose, frame, False, values[1], values[2]) 
            
            #frames = cv2.putText(frame,f"{frame_no} {label}", (50,50), cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2, cv2.LINE_AA)
            if hp:
                cv2.imwrite("ExtremeHeadPose/frame_%d.jpg" % frame_no, frame)
            else:
                gd = check_gaze(res_mesh, frame, False) #check_gaze(p_mesh,p_frame, p_check=False, p_center_value=None)
                #cv2.putText(frames, "Center", (50,50), cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2, cv2.LINE_AA)
                if gd:
                    cv2.imshow("Center",frames)
                    cv2.waitKey(0)
                else: continue
            #cv2.imshow("Head pose", frames)
            #cv2.waitKey(0)
    else:
        
        continue
    if cv2.waitKey(1) & 0xFF == 27:
        break
        
cv2.destroyAllWindows()       

In [ ]:
skip_rate = 1
frame_no = 0 
#video_path = "C:\\Users\\99455\\Desktop\\DataScience\\Proctoring Task\\testvideo6.mp4"
video_path =  'C:/Users/99455/Desktop/DataScience/OEP database/subject1/Yousef1.avi'


video = VideoHandler(video_path)
results = video.process_video()
    
print(results)
    

In [ ]:
if horizontal_ratio >= vertical_ratio:
            frame =  self.__draw_lines(p_frame, nose_2d, leye_2d, mbrow_2d)
            frame =  self.__draw_lines(frame, nose_2d, reye_2d, mbrow_2d)
        else:
            frame = self.__draw_lines(p_frame, mchin_2d, lnose_2d, rnose_2d)
            frame = self.__draw_lines(frame, mbrow_2d, lnose_2d, rnose_2d)

In [ ]:


sequence = [] 
whole_sequence = []
i = 0
for item in results:  
    i+=1
    if item[0] == True:     
        sequence.append((item[1],item[3]))
        if len(sequence) == len(results):
            whole_sequence.append(sequence)
        elif i == len(results) and len(sequence) > 5:
            whole_sequence.append(sequence)
    else:  
        if len(sequence) > 10:
            whole_sequence.append(sequence)
        sequence = []
def calculate_diff(curr, prev):
    return (curr-prev ) < 3000
prev = None
for seq in whole_sequence:
    mill_sec_begin = seq[0][0]
    begin = make_timestamp(mill_sec_begin)
    mill_sec_end = seq[-1][0]
    end = make_timestamp(mill_sec_end)
    together = begin + " - " + end + " " + str(seq[-1][1])
    #print(together)
#     current = mill_sec_begin
#     array[]
#     if prev is not None:
#         diff = calculate_diff(current, prev)
#         if diff:
#             son1 = mill_sec_end
#             array[prev][1] = son1
            
#         else:
#             prev = None
#     else:
#         son1 = mill_sec_end
#         bas1 = mill_sec_begin
#         together = make_timestamp(bas1) + " - " + make_timestamp(son1) + " " + str(seq[-1][1])
#         print(together)
#     prev = mill_sec_end
# print(together)
#print("\n") 
i = 0
for seq in whole_sequence:
    i += 1
    begin = seq[0][0]
    end   = seq [-1][0]
    curr  = begin
    if prev is not None:
        if (curr - prev) < 3000:
            s2 = seq[-1][0]
        else:
            together = make_timestamp(b1) + " - " + make_timestamp(s2) + " " + str(seq[-1][1])
            b1 = begin
            s2 = end
            print(together)
            if i == len(whole_sequence):
                together = make_timestamp(b1) + " - " + make_timestamp(s2) + " " + str(seq[-1][1])
                #print(together)
                
    else:
        #keep_b1 = begin
        #b1 = keep_b1
        b1 = begin
        
    prev = end
    curr_begin = begin

                  
    
    #print(together)

In [ ]:
self.face_pose = mp.solutions.pose.Pose(static_image_mode=True, model_complexity = 1, min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [ ]:
#self.mp_face_detection = mp.solutions.face_detection
        #self.face_detection = mp.solutions.face_detection.FaceDetection(min_detection_confidence=0.5)
        #self.video_path = video_path
self.mp_drawing_styles = mp.solutions.drawing_styles
self.drawing_spec = mp.solutions.drawing_utils.DrawingSpec(thickness=1,circle_radius=1)